In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 199 kB 43.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=561b0a155daaae3b9b832463988f29dfca90410efe6ef0cf3e615850d8a6a751
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[4]").appName("technique").getOrCreate()
df_date=spark.read.csv("/content/date.csv",header=True,inferSchema=True)
df_date.show()

+-------------------+
|         input_date|
+-------------------+
|2021-12-18 16:20:08|
+-------------------+



#format date in pyspark

In [ ]:
from pyspark.sql import functions as f
(df_date.withColumn("to_date",f.to_date("input_date"))
.withColumn("to_timestamp",f.to_timestamp("input_date"))
.withColumn("dd/MM/yyyy",f.date_format("input_date","dd/MM/yyyy"))
.withColumn("dd/MMM/yyyy",f.date_format("input_date","dd/MMM/yyyy"))
.withColumn("day",f.date_format("input_date","E")).show())

+-------------------+----------+-------------------+----------+-----------+---+
|         input_date|   to_date|       to_timestamp|dd/MM/yyyy|dd/MMM/yyyy|day|
+-------------------+----------+-------------------+----------+-----------+---+
|2021-12-18 16:20:08|2021-12-18|2021-12-18 16:20:08|18/12/2021|18/Dec/2021|Sat|
+-------------------+----------+-------------------+----------+-----------+---+



#arithmetic operations on date

In [ ]:
(df_date.withColumn("to_date",f.to_date("input_date"))
.withColumn("current_date",f.current_date())
.withColumn("datediff",f.datediff(f.col("current_date"),f.col("to_date"))).show())

+-------------------+----------+------------+--------+
|         input_date|   to_date|current_date|datediff|
+-------------------+----------+------------+--------+
|2021-12-18 16:20:08|2021-12-18|  2022-07-20|     214|
+-------------------+----------+------------+--------+



In [ ]:
(df_date.withColumn("to_date",f.to_date("input_date"))
.withColumn("current_date",f.current_date())
.withColumn("datediff",f.datediff(f.col("current_date"),f.col("to_date")))
.withColumn("curdateadd",f.date_add(f.col("current_date"),5))
.withColumn("curdatesub",f.date_sub(f.col("current_date"),5)).show())

+-------------------+----------+------------+--------+----------+----------+
|         input_date|   to_date|current_date|datediff|curdateadd|curdatesub|
+-------------------+----------+------------+--------+----------+----------+
|2021-12-18 16:20:08|2021-12-18|  2022-07-20|     214|2022-07-25|2022-07-15|
+-------------------+----------+------------+--------+----------+----------+



#extract date parts

In [ ]:
(df_date.withColumn("to_date",f.to_date("input_date"))
.withColumn("year",f.year("to_date"))
.withColumn("Quarter",f.quarter("to_date"))
.withColumn("month",f.month("to_date"))
.withColumn("week",f.weekofyear("to_date"))
.withColumn("day",f.dayofyear("to_date"))
.withColumn("dayofmonth",f.dayofmonth("to_date"))
.withColumn("dayofweek",f.dayofweek("to_date")).show())

+-------------------+----------+----+-------+-----+----+---+----------+---------+
|         input_date|   to_date|year|Quarter|month|week|day|dayofmonth|dayofweek|
+-------------------+----------+----+-------+-----+----+---+----------+---------+
|2021-12-18 16:20:08|2021-12-18|2021|      4|   12|  50|352|        18|        7|
+-------------------+----------+----+-------+-----+----+---+----------+---------+



#extract time parts

In [ ]:
(df_date.withColumn("to_timestamp",f.to_timestamp("input_date"))
.withColumn("hour",f.hour("input_date"))
.withColumn("min",f.minute("input_date"))
.withColumn("second",f.second("to_timestamp"))).show()#or .withColumn("hour",f.hour("input_date"))

+-------------------+-------------------+----+---+------+
|         input_date|       to_timestamp|hour|min|second|
+-------------------+-------------------+----+---+------+
|2021-12-18 16:20:08|2021-12-18 16:20:08|  16| 20|     8|
+-------------------+-------------------+----+---+------+



#window functions

In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[4]").appName("technique").getOrCreate()
df_cricket=spark.read.csv("/content/cricket.csv",header=True,inferSchema=True)
df_cricket.show()

+---+-------+-------+-----+
| id|   name|country|score|
+---+-------+-------+-----+
|  1|  Rohit|  India|   80|
|  2|  Virat|  India|   90|
|  3|  Rahul|  India|   80|
|  4|   Pant|  India|   60|
|  5|  Gayle|     WI|   80|
|  6|Pollard|     WI|   70|
|  7|   Kane|     NZ|   85|
|  8| Taylor|     NZ|   95|
+---+-------+-------+-----+



overall player,rownumber,ranknumber

#create window for overall/entire data in descending order of score..

In [ ]:
from pyspark.sql.window import Window
full_window=Window.orderBy(f.desc("score"))

In [ ]:
(df_cricket.withColumn("row_number",f.row_number().over(full_window))
.withColumn("rank",f.rank().over(full_window))).withColumn("dense_rank",f.dense_rank().over(full_window)).show()

+---+-------+-------+-----+----------+----+----------+
| id|   name|country|score|row_number|rank|dense_rank|
+---+-------+-------+-----+----------+----+----------+
|  8| Taylor|     NZ|   95|         1|   1|         1|
|  2|  Virat|  India|   90|         2|   2|         2|
|  7|   Kane|     NZ|   85|         3|   3|         3|
|  1|  Rohit|  India|   80|         4|   4|         4|
|  3|  Rahul|  India|   80|         5|   4|         4|
|  5|  Gayle|     WI|   80|         6|   4|         4|
|  6|Pollard|     WI|   70|         7|   7|         5|
|  4|   Pant|  India|   60|         8|   8|         6|
+---+-------+-------+-----+----------+----+----------+



Rank will skip number for same score(skipped 5&6)..
dense_rank willnot skipped same score..

#country wise players with row &rank,dense_rank

In [ ]:
country=Window.partitionBy("country").orderBy(f.desc("score"))

In [ ]:
(df_cricket.withColumn("row_number",f.row_number().over(country))
.withColumn("rank",f.rank().over(country))).withColumn("dense_rank",f.dense_rank().over(country)).show()

+---+-------+-------+-----+----------+----+----------+
| id|   name|country|score|row_number|rank|dense_rank|
+---+-------+-------+-----+----------+----+----------+
|  2|  Virat|  India|   90|         1|   1|         1|
|  1|  Rohit|  India|   80|         2|   2|         2|
|  3|  Rahul|  India|   80|         3|   2|         2|
|  4|   Pant|  India|   60|         4|   4|         3|
|  8| Taylor|     NZ|   95|         1|   1|         1|
|  7|   Kane|     NZ|   85|         2|   2|         2|
|  5|  Gayle|     WI|   80|         1|   1|         1|
|  6|Pollard|     WI|   70|         2|   2|         2|
+---+-------+-------+-----+----------+----+----------+



In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[4]").appName("technique").getOrCreate()
df_array=spark.read.csv("/content/array.csv",header=True,inferSchema=True)
df_array.show()

+-----+--------------------+
| name|                data|
+-----+--------------------+
|  Tom|      10|40|20|10|30|
|Jenny|100|800|200|100|3...|
+-----+--------------------+



#create a column as arraytype

In [ ]:
dfA=df_array.withColumn("array_type",f.split("data","\|"))
dfA.show(truncate=False)

+-----+-------------------------------+----------------------------------------+
|name |data                           |array_type                              |
+-----+-------------------------------+----------------------------------------+
|Tom  |10|40|20|10|30                 |[10, 40, 20, 10, 30]                    |
|Jenny|100|800|200|100|300|200|500|300|[100, 800, 200, 100, 300, 200, 500, 300]|
+-----+-------------------------------+----------------------------------------+



#executions on array_type column
 

#1.size

In [ ]:
dfA.withColumn("size",f.size("array_type")).select("name","array_type","size").show(truncate=False)

+-----+----------------------------------------+----+
|name |array_type                              |size|
+-----+----------------------------------------+----+
|Tom  |[10, 40, 20, 10, 30]                    |5   |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|8   |
+-----+----------------------------------------+----+



#2.array_min

In [ ]:
dfA.withColumn("array_min",f.array_min("array_type")).select("name","array_type","array_min").show(truncate=False)

+-----+----------------------------------------+---------+
|name |array_type                              |array_min|
+-----+----------------------------------------+---------+
|Tom  |[10, 40, 20, 10, 30]                    |10       |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|100      |
+-----+----------------------------------------+---------+



#3.array_max

In [ ]:
dfA.withColumn("array_max",f.array_max("array_type")).select("name","array_type","array_max").show(truncate=False)

+-----+----------------------------------------+---------+
|name |array_type                              |array_max|
+-----+----------------------------------------+---------+
|Tom  |[10, 40, 20, 10, 30]                    |40       |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|800      |
+-----+----------------------------------------+---------+



#4.element_at,array_contains,array_position

In [ ]:
(dfA.withColumn("element_at",f.element_at("array_type",2))
.withColumn("array_contains",f.array_contains("array_type","200")).select("name","array_type","element_at","array_contains")).show(truncate=False)
#.withColumn("array_positions",f.array_position("array_type","300")).select("name","array_type","element_at","array_contains","array_position"))

+-----+----------------------------------------+----------+--------------+
|name |array_type                              |element_at|array_contains|
+-----+----------------------------------------+----------+--------------+
|Tom  |[10, 40, 20, 10, 30]                    |40        |false         |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|800       |true          |
+-----+----------------------------------------+----------+--------------+



#5.Reverse arraycolumn

In [ ]:
dfA.withColumn("Reverse",f.reverse("array_type")).select("name","array_type","Reverse").show(truncate=False)

+-----+----------------------------------------+----------------------------------------+
|name |array_type                              |Reverse                                 |
+-----+----------------------------------------+----------------------------------------+
|Tom  |[10, 40, 20, 10, 30]                    |[30, 10, 20, 40, 10]                    |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|[300, 500, 200, 300, 100, 200, 800, 100]|
+-----+----------------------------------------+----------------------------------------+



#6.Sorting array column

In [ ]:
dfA.withColumn("Sort",f.array_sort("array_type")).select("name","array_type","Sort").show(truncate=False)

+-----+----------------------------------------+----------------------------------------+
|name |array_type                              |Sort                                    |
+-----+----------------------------------------+----------------------------------------+
|Tom  |[10, 40, 20, 10, 30]                    |[10, 10, 20, 30, 40]                    |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|[100, 100, 200, 200, 300, 300, 500, 800]|
+-----+----------------------------------------+----------------------------------------+



#7.distinctelements in array_column

In [ ]:
(dfA.withColumn("distinct",f.array_distinct("array_type")).select("name","array_type","distinct")).show(truncate=False)

+-----+----------------------------------------+-------------------------+
|name |array_type                              |distinct                 |
+-----+----------------------------------------+-------------------------+
|Tom  |[10, 40, 20, 10, 30]                    |[10, 40, 20, 30]         |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|[100, 800, 200, 300, 500]|
+-----+----------------------------------------+-------------------------+



#8.join elements

In [ ]:
dfA.withColumn("joins",f.array_join("array_type","--")).select("name","array_type","joins").show(truncate=False)

+-----+----------------------------------------+--------------------------------------+
|name |array_type                              |joins                                 |
+-----+----------------------------------------+--------------------------------------+
|Tom  |[10, 40, 20, 10, 30]                    |10--40--20--10--30                    |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|100--800--200--100--300--200--500--300|
+-----+----------------------------------------+--------------------------------------+



#9.explode array elements into records

In [ ]:
dfA.withColumn("Explode",f.explode("array_type")).select("name","array_type","Explode").show(truncate=False)

+-----+----------------------------------------+-------+
|name |array_type                              |Explode|
+-----+----------------------------------------+-------+
|Tom  |[10, 40, 20, 10, 30]                    |10     |
|Tom  |[10, 40, 20, 10, 30]                    |40     |
|Tom  |[10, 40, 20, 10, 30]                    |20     |
|Tom  |[10, 40, 20, 10, 30]                    |10     |
|Tom  |[10, 40, 20, 10, 30]                    |30     |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|100    |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|800    |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|200    |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|100    |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|300    |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|200    |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|500    |
|Jenny|[100, 800, 200, 100, 300, 200, 500, 300]|300    |
+-----+----------------------------------------+-------+



In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[4]").appName("technique").getOrCreate()
df_s=spark.read.csv("/content/string.csv",header=True,inferSchema=True)
df_s.show(truncate=False)

+---+-----+---------------------------+
|id |code |description                |
+---+-----+---------------------------+
|1  |AXD  |       Wide  voltage  range|
|2  |EM   |   Full  ISI compliance    |
|3  |OUEWS|HIGH efficiency            |
|4  |NHD  |Lightning proof            |
|5  |LEJ  |     Weather proof         |
+---+-----+---------------------------+



#Find the length of string

In [ ]:
df_s.withColumn("length",f.length("description")).select("id","description","length").show(truncate=False)

+---+---------------------------+------+
|id |description                |length|
+---+---------------------------+------+
|1  |       Wide  voltage  range|27    |
|2  |   Full  ISI compliance    |23    |
|3  |HIGH efficiency            |15    |
|4  |Lightning proof            |15    |
|5  |     Weather proof         |18    |
+---+---------------------------+------+



#case conversion ofstring

In [ ]:
df_s.withColumn("upper",f.upper("description")).withColumn("lower",f.lower("description")).withColumn("initcap",f.initcap("description")).select("id","description","upper","lower","initcap").show(truncate=False)

+---+---------------------------+---------------------------+---------------------------+---------------------------+
|id |description                |upper                      |lower                      |initcap                    |
+---+---------------------------+---------------------------+---------------------------+---------------------------+
|1  |       Wide  voltage  range|       WIDE  VOLTAGE  RANGE|       wide  voltage  range|       Wide  Voltage  Range|
|2  |   Full  ISI compliance    |   FULL  ISI COMPLIANCE    |   full  isi compliance    |   Full  Isi Compliance    |
|3  |HIGH efficiency            |HIGH EFFICIENCY            |high efficiency            |High Efficiency            |
|4  |Lightning proof            |LIGHTNING PROOF            |lightning proof            |Lightning Proof            |
|5  |     Weather proof         |     WEATHER PROOF         |     weather proof         |     Weather Proof         |
+---+---------------------------+-----------------------

#remove whitespaces

In [ ]:
df_s.withColumn("ltrim",f.ltrim("description")).withColumn("rtrim",f.rtrim("description")).withColumn("trim",f.trim("description")).select("id","description","rtrim","ltrim","trim").show(truncate=False)

+---+---------------------------+---------------------------+--------------------+--------------------+
|id |description                |rtrim                      |ltrim               |trim                |
+---+---------------------------+---------------------------+--------------------+--------------------+
|1  |       Wide  voltage  range|       Wide  voltage  range|Wide  voltage  range|Wide  voltage  range|
|2  |   Full  ISI compliance    |   Full  ISI compliance    |Full  ISI compliance|Full  ISI compliance|
|3  |HIGH efficiency            |HIGH efficiency            |HIGH efficiency     |HIGH efficiency     |
|4  |Lightning proof            |Lightning proof            |Lightning proof     |Lightning proof     |
|5  |     Weather proof         |     Weather proof         |Weather proof       |Weather proof       |
+---+---------------------------+---------------------------+--------------------+--------------------+



#find the location of substring in string

In [ ]:
df_s.withColumn("locate",f.locate("proof","description")).show(truncate=False)

+---+-----+---------------------------+------+
|id |code |description                |locate|
+---+-----+---------------------------+------+
|1  |AXD  |       Wide  voltage  range|0     |
|2  |EM   |   Full  ISI compliance    |0     |
|3  |OUEWS|HIGH efficiency            |0     |
|4  |NHD  |Lightning proof            |11    |
|5  |LEJ  |     Weather proof         |14    |
+---+-----+---------------------------+------+



#getting substring


In [ ]:
df_s.withColumn("substring",f.substring("description",5,10)).show(truncate=False)#col,pos,len

+---+-----+---------------------------+----------+
|id |code |description                |substring |
+---+-----+---------------------------+----------+
|1  |AXD  |       Wide  voltage  range|   Wide  v|
|2  |EM   |   Full  ISI compliance    |ull  ISI c|
|3  |OUEWS|HIGH efficiency            | efficienc|
|4  |NHD  |Lightning proof            |tning proo|
|5  |LEJ  |     Weather proof         | Weather p|
+---+-----+---------------------------+----------+



#replace substring

In [ ]:
df_s.withColumn("replace_exp",f.regexp_replace("description","proof","hello")).show(truncate=False)

+---+-----+---------------------------+---------------------------+
|id |code |description                |replace_exp                |
+---+-----+---------------------------+---------------------------+
|1  |AXD  |       Wide  voltage  range|       Wide  voltage  range|
|2  |EM   |   Full  ISI compliance    |   Full  ISI compliance    |
|3  |OUEWS|HIGH efficiency            |HIGH efficiency            |
|4  |NHD  |Lightning proof            |Lightning hello            |
|5  |LEJ  |     Weather proof         |     Weather hello         |
+---+-----+---------------------------+---------------------------+



#covert string into fixedlen (padding)

In [ ]:
df_s.withColumn("rpad",f.rpad("code",5,"$")).withColumn("lpad",f.lpad("code",10,"#")).select("code","rpad","lpad").show(truncate=False)

+-----+-----+----------+
|code |rpad |lpad      |
+-----+-----+----------+
|AXD  |AXD$$|#######AXD|
|EM   |EM$$$|########EM|
|OUEWS|OUEWS|#####OUEWS|
|NHD  |NHD$$|#######NHD|
|LEJ  |LEJ$$|#######LEJ|
+-----+-----+----------+



In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[4]").appName("technique").getOrCreate()
dfu=spark.read.csv("/content/udf_input.csv",header=True,inferSchema=True)
dfu.show(truncate=False)

+---+-------+--------+
|id |name   |order   |
+---+-------+--------+
|1  |Tom    |T-shirt |
|2  |Jenny  |shoes   |
|3  |Andrew |Grocery |
|4  |Michael|Book    |
|5  |Jasmine|keyboard|
+---+-------+--------+



#user defined functions

In [ ]:
def display_msg(name,order):
                msg=f"""Hello {name}, your order {order} will be delivered today."""
                return msg

In [ ]:
print(display_msg("sai","watch"))

Hello sai, your order watch will be delivered today.


In [ ]:
udf_display_msg=f.udf(display_msg)

In [ ]:
(dfu.withColumn("message",udf_display_msg(f.col('name'),f.col("order"))).show(truncate=False))

+---+-------+--------+-----------------------------------------------------------+
|id |name   |order   |message                                                    |
+---+-------+--------+-----------------------------------------------------------+
|1  |Tom    |T-shirt |Hello Tom, your order T-shirt will be delivered today.     |
|2  |Jenny  |shoes   |Hello Jenny, your order shoes will be delivered today.     |
|3  |Andrew |Grocery |Hello Andrew, your order Grocery will be delivered today.  |
|4  |Michael|Book    |Hello Michael, your order Book will be delivered today.    |
|5  |Jasmine|keyboard|Hello Jasmine, your order keyboard will be delivered today.|
+---+-------+--------+-----------------------------------------------------------+

